# Function for training set

In [3]:
import os
import shutil

source_dir = '/Users/rajdipingale/Downloads/archive (15)/images_128_128'

destination_dir = '/Users/rajdipingale/Downloads/archive (15)/train'

os.makedirs(destination_dir, exist_ok=True)

for subdir, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            file_path = os.path.join(subdir, file)
            
            shutil.move(file_path, destination_dir)

print("All images have been moved to:", destination_dir)


All images have been moved to: /Users/rajdipingale/Downloads/archive (15)/train


# VAE Architecture

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from PIL import Image

def load_image_dataset(data_dir, img_size=(128, 128), batch_size=1024):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        label_mode=None,  # No labels needed for VAE
        image_size=img_size,
        batch_size=batch_size
    )
    return dataset

data_dir = '/Users/rajdipingale/Downloads/archive (15)/train'
img_size = (128, 128)  
batch_size = 32  
train_dataset = load_image_dataset(data_dir, img_size=img_size, batch_size=batch_size)

def normalize_image(image):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

train_dataset = train_dataset.map(normalize_image)

latent_dim = 512

# VAE encoder network for RGB images (128x128)
encoder_inputs = keras.Input(shape=(128, 128, 3))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)  
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)  
x = layers.Flatten()(x)
x = layers.Dense(16, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)  
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)  
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var], name='encoder')

# Sampler
class Sampler(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Sampler()([z_mean, z_log_var])

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16 * 16 * 128, activation='relu')(latent_inputs)  
x = layers.Reshape((16, 16, 128))(x)  # Reshape the latent vector
x = layers.Conv2DTranspose(128, 3, activation='relu', strides=2, padding='same')(x)  
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)   
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)   
decoder_outputs = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)  
decoder = keras.Model(latent_inputs, decoder_outputs, name='decoder')




# VAE model
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.sampler = Sampler()
        self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = keras.metrics.Mean(name='reconstruction_loss')
        self.kl_loss_tracker = keras.metrics.Mean(name='kl_loss')

    @property
    def metrics(self):
        return [self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encoder(data)
            z = self.sampler([z_mean, z_log_var])
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.mean_squared_error(data, reconstruction)
            )
            kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            'total_loss': self.total_loss_tracker.result(),
            'reconstruction_loss': self.reconstruction_loss_tracker.result(),
            'kl_loss': self.kl_loss_tracker.result(),
        }

# Create and train the VAE
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(train_dataset, epochs=1, batch_size=1024)

n = 10  
figure = np.zeros((128 * n, 128 * n, 3), dtype=np.uint8)

grid_x = np.linspace(-2, 2, n)
grid_y = np.linspace(-2, 2, n)[::-1]

for i, yi in enumerate(grid_y):
    for j, xi in enumerate(grid_x):
        z_sample = np.random.normal(size=(1, latent_dim))  
        x_decoded = vae.decoder.predict(z_sample)  
        digit = (x_decoded[0] * 255).astype(np.uint8)  
        figure[i * 128: (i + 1) * 128, j * 128: (j + 1) * 128, :] = digit  

plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.axis("off")
plt.show()
